# Chapter 9: Functions, Expressions, and Common Patterns

PostgreSQL's rich function library and advanced SQL features enable complex data transformations directly within the database. This chapter covers essential functions for data manipulation, conditional logic, recursive queries, and lateral joins—providing industry-standard patterns for solving complex problems while maintaining performance and readability.

## 9.1 Conditional Expressions and Logic

### 9.1.1 CASE Expressions (Searched and Simple)

`CASE` provides conditional logic within SQL statements, eliminating the need for application-layer branching.

```sql
-- Searched CASE (most common, uses boolean conditions)
SELECT 
    order_id,
    total_amount,
    CASE 
        WHEN total_amount > 1000 THEN 'high_value'
        WHEN total_amount > 500 THEN 'medium_value'
        ELSE 'standard'
    END as value_tier,
    CASE 
        WHEN status = 'shipped' AND delivered_at IS NULL THEN 'in_transit'
        WHEN status = 'shipped' AND delivered_at IS NOT NULL THEN 'delivered'
        ELSE status
    END as display_status
FROM orders;

-- Simple CASE (compares single expression to values)
SELECT 
    order_id,
    CASE status
        WHEN 'pending' THEN 'Awaiting Payment'
        WHEN 'processing' THEN 'Preparing Order'
        WHEN 'shipped' THEN 'On the Way'
        WHEN 'delivered' THEN 'Completed'
        ELSE 'Unknown'
    END as status_label
FROM orders;

-- CASE in aggregates (pivot-like operations)
SELECT 
    DATE_TRUNC('month', created_at) as month,
    COUNT(*) as total_orders,
    COUNT(CASE WHEN status = 'completed' THEN 1 END) as completed_orders,
    COUNT(CASE WHEN status = 'cancelled' THEN 1 END) as cancelled_orders,
    ROUND(
        COUNT(CASE WHEN status = 'completed' THEN 1 END) * 100.0 / COUNT(*), 
        2
    ) as completion_rate
FROM orders
GROUP BY 1
ORDER BY 1;

-- Industry tip: Use FILTER clause (PostgreSQL 9.4+) instead of CASE in aggregates
-- More readable, same performance
SELECT 
    DATE_TRUNC('month', created_at) as month,
    COUNT(*) as total,
    COUNT(*) FILTER (WHERE status = 'completed') as completed,
    COUNT(*) FILTER (WHERE status = 'cancelled') as cancelled
FROM orders
GROUP BY 1;

-- CASE with ELSE NULL (explicit vs implicit)
-- Without ELSE, CASE returns NULL if no condition matches
SELECT 
    product_name,
    CASE 
        WHEN stock_quantity = 0 THEN 'out_of_stock'
        WHEN stock_quantity < 10 THEN 'low_stock'
        -- Implicit ELSE NULL (returns NULL if stock >= 10)
    END as stock_status
FROM products;

-- Always use ELSE for safety in production code
SELECT 
    product_name,
    CASE 
        WHEN stock_quantity = 0 THEN 'out_of_stock'
        WHEN stock_quantity < 10 THEN 'low_stock'
        ELSE 'in_stock'
    END as stock_status
FROM products;
```

### 9.1.2 COALESCE and NULL Handling Patterns

`COALESCE` returns the first non-NULL value from its arguments, essential for providing defaults and handling missing data.

```sql
-- Basic default values
SELECT 
    user_id,
    COALESCE(display_name, full_name, email, 'Anonymous') as identifier
FROM users;
-- Returns first non-NULL: display_name preferred, then full_name, then email, then 'Anonymous'

-- Fallback chain for configuration
SELECT 
    COALESCE(
        user_preference_timezone,  -- User-specific
        organization_default_timezone,  -- Organization default
        'UTC'  -- System default
    ) as effective_timezone
FROM user_settings;

-- Safe division (avoid division by zero)
SELECT 
    product_id,
    revenue,
    orders,
    COALESCE(revenue / NULLIF(orders, 0), 0) as avg_order_value
FROM sales_summary;
-- NULLIF converts 0 to NULL, preventing division by zero
-- COALESCE converts NULL result to 0

-- Combining with aggregates (handle no rows)
SELECT 
    COALESCE(MAX(updated_at), '1970-01-01'::timestamptz) as last_update
FROM change_log
WHERE table_name = 'users';
-- Returns epoch if no rows found, instead of NULL

-- COALESCE vs ISNULL vs NVL
-- PostgreSQL: COALESCE (ANSI standard, accepts 2+ args)
-- SQL Server: ISNULL (non-standard, 2 args only)
-- Oracle: NVL (non-standard)
-- Always use COALESCE for portability

-- Performance consideration: COALESCE evaluates left-to-right
-- Put most likely non-NULL values first
SELECT COALESCE(frequently_populated, rarely_populated) FROM table;
-- Better than reverse order (avoids evaluating second arg often)
```

### 9.1.3 NULLIF and Conditional NULLs

`NULLIF` returns NULL if two expressions are equal, useful for suppressing specific values and preventing division by zero.

```sql
-- Suppress default/placeholder values
SELECT 
    NULLIF(phone_number, '000-000-0000') as clean_phone
FROM users;
-- Returns NULL if phone is the placeholder, else returns actual number

-- Normalize empty strings to NULL
SELECT 
    NULLIF(TRIM(email), '') as normalized_email
FROM contacts;
-- Empty string becomes NULL (better for indexing and comparisons)

-- Prevent division by zero (pattern)
SELECT 
    numerator,
    denominator,
    numerator / NULLIF(denominator, 0) as safe_ratio
FROM metrics;
-- If denominator is 0, expression becomes NULL (not error)

-- NULLIF with aggregates (ignore specific values)
SELECT 
    AVG(NULLIF(rating, 0)) as avg_rating_excluding_zeros
FROM reviews;
-- Excludes 0-star ratings from average calculation

-- Combined NULLIF and COALESCE pattern
SELECT 
    COALESCE(
        NULLIF(TRIM(notes), ''),
        'No notes provided'
    ) as display_notes
FROM tickets;
-- Empty string -> NULL -> 'No notes provided'
```

### 9.1.4 GREATEST and LEAST (Multi-Value Comparisons)

```sql
-- Clamp values to range
SELECT 
    user_id,
    GREATEST(0, LEAST(100, completion_percentage)) as bounded_percentage
FROM progress;
-- Ensures value between 0 and 100

-- Effective dates (latest start, earliest end)
SELECT 
    GREATEST(campaign_start, subscription_start) as effective_start,
    LEAST(campaign_end, subscription_end) as effective_end
FROM campaigns;

-- Safety limits
UPDATE accounts 
SET balance = LEAST(balance, 1000000)  -- Cap at 1M
WHERE balance > 1000000;
```

## 9.2 String Manipulation and Pattern Matching

### 9.2.1 String Functions (ANSI and PostgreSQL-Specific)

```sql
-- Concatenation (|| operator preferred over CONCAT function)
SELECT first_name || ' ' || last_name as full_name FROM users;
-- Standard SQL, returns NULL if any operand is NULL

-- Safe concatenation with NULL handling
SELECT CONCAT(first_name, ' ', last_name) as full_name FROM users;
-- CONCAT treats NULL as empty string (PostgreSQL 9.1+)

-- Formatting (printf-style)
SELECT format('Hello %s, you have %s messages', username, msg_count) FROM users;
-- Type-safe formatting, handles NULLs as empty string

-- String length and manipulation
SELECT 
    email,
    LENGTH(email) as char_count,
    CHAR_LENGTH(email) as same_as_length,
    OCTET_LENGTH(email) as bytes,  -- Important for UTF-8 multi-byte chars
    LEFT(email, 3) as domain_prefix,
    RIGHT(email, 10) as domain_suffix,
    SUBSTRING(email FROM 1 FOR 5) as first_five,  -- SQL standard
    SUBSTR(email, 1, 5) as same_thing,  -- Oracle compatibility
    TRIM(BOTH ' ' FROM email) as clean_email,
    LTRIM(rpad(email, 20, '*')) as padded_left  -- Pad then trim for masking
FROM contacts;

-- Position and substring extraction
SELECT 
    email,
    POSITION('@' IN email) as at_position,
    STRPOS(email, '@') as same_thing,  -- PostgreSQL specific
    SUBSTRING(email FROM POSITION('@' IN email) + 1) as domain,
    SPLIT_PART(email, '@', 2) as domain_part  -- Split into parts (1-based)
FROM users;

-- Replace and translate
SELECT 
    phone,
    REPLACE(phone, '-', '') as digits_only,
    TRANSLATE(phone, '()-', '') as same_thing,  -- Character-by-character replacement
    REGEXP_REPLACE(phone, '[^0-9]', '', 'g') as regex_digits_only
FROM contacts;

-- Case conversion
SELECT 
    LOWER(email) as lower_email,  -- For case-insensitive comparison
    UPPER(sku) as upper_sku,      -- Normalized product codes
    INITCAP(full_name) as title_case_name  -- "john doe" -> "John Doe"
FROM users;

-- Padding (LPAD/RPAD)
SELECT 
    LPAD(invoice_number::TEXT, 10, '0') as padded_invoice,
    -- '123' -> '0000000123'
    RPAD(status, 10, '.') as status_dots
    -- 'active' -> 'active....'
FROM invoices;
```

### 9.2.2 Regular Expressions (POSIX and Advanced)

PostgreSQL supports POSIX regular expressions with multiple matching modes.

```sql
-- Basic pattern matching (~ case sensitive, ~* case insensitive)
SELECT * FROM users WHERE email ~ '@example\.com$';  -- Ends with @example.com
SELECT * FROM users WHERE email ~* '@EXAMPLE\.COM$'; -- Case insensitive

-- Negative matching (!~ and !~*)
SELECT * FROM users WHERE email !~ '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$';

-- Regexp extraction (return matching substring)
SELECT 
    SUBSTRING(description FROM 'price: (\d+\.?\d*)') as price_string
FROM products
WHERE description ~ 'price: \d';

-- Regexp replace (cleaning data)
UPDATE users 
SET phone = REGEXP_REPLACE(phone, '[^0-9]', '', 'g')
WHERE phone ~ '[^0-9]';
-- 'g' flag = replace all occurrences (global)

-- Advanced replacement with backreferences
SELECT 
    REGEXP_REPLACE(phone, '(\d{3})(\d{3})(\d{4})', '(\1) \2-\3') as formatted
FROM users;
-- '5551234567' -> '(555) 123-4567'

-- Regexp split (explode string to rows)
SELECT * FROM REGEXP_SPLIT_TO_TABLE('apple,banana,orange', ',');
-- Returns 3 rows: apple, banana, orange

SELECT REGEXP_SPLIT_TO_ARRAY('apple,banana,orange', ',');
-- Returns array: {apple,banana,orange}

-- Regexp match groups (extract multiple groups)
SELECT 
    (REGEXP_MATCHES(email, '^([^@]+)@(.+)$'))[1] as username,
    (REGEXP_MATCHES(email, '^([^@]+)@(.+)$'))[2] as domain
FROM users;
-- Returns array of captured groups

-- Pattern matching flags
SELECT * FROM logs WHERE message ~* 'error|warning|critical';  -- Case insensitive OR

-- Word boundaries
SELECT * FROM articles WHERE content ~ '\yPostgreSQL\y';  -- \y = word boundary
-- Matches "PostgreSQL" but not "PostgreSQLs" or "MyPostgreSQL"

-- Industry standard: Use SIMILAR TO sparingly (SQL standard but limited)
-- Prefer ~ and ~* operators with POSIX regex (more powerful)
```

### 9.2.3 String Aggregation (STRING_AGG)

Aggregating rows into delimited strings is a common reporting requirement.

```sql
-- Basic aggregation
SELECT 
    department_id,
    STRING_AGG(employee_name, ', ' ORDER BY employee_name) as team_members
FROM employees
GROUP BY department_id;

-- With DISTINCT (PostgreSQL 9.0+)
SELECT 
    category_id,
    STRING_AGG(DISTINCT tag, '|') as unique_tags
FROM products
GROUP BY category_id;

-- Handling NULLs (filter them out first)
SELECT 
    order_id,
    STRING_AGG(note, '; ' ORDER BY created_at) as order_notes
FROM order_notes
WHERE note IS NOT NULL  -- NULLs would cause entire result to be NULL
GROUP BY order_id;

-- Alternative: COALESCE inside aggregation
SELECT 
    order_id,
    STRING_AGG(COALESCE(note, '[empty]'), '; ') as notes_with_placeholders
FROM order_notes
GROUP BY order_id;

-- JSON-style aggregation (array to string)
SELECT 
    user_id,
    '[' || STRING_AGG('"' || permission || '"', ',') || ']' as json_array
FROM user_permissions
GROUP BY user_id;
-- Returns: ["read", "write", "admin"]

-- Better: Use JSON_AGG (see JSON section)
```

## 9.3 Date and Time Functions

### 9.3.1 Current Timestamp Functions (Critical Differences)

Understanding the distinction between transaction time and statement time is crucial for consistency.

```sql
-- Transaction timestamp (constant throughout transaction)
SELECT NOW();           -- TIMESTAMPTZ, same for all calls in transaction
SELECT CURRENT_TIMESTAMP; -- Same as NOW()
SELECT TRANSACTION_TIMESTAMP(); -- Explicit alias for NOW()

-- Statement timestamp (when statement began execution)
SELECT STATEMENT_TIMESTAMP(); -- Slightly different from NOW() in long transactions

-- Clock timestamp (actual wall clock time, changes during statement)
SELECT CLOCK_TIMESTAMP();  -- Different each call, even within single statement

-- Current date and time (session timezone)
SELECT CURRENT_DATE;       -- DATE type
SELECT CURRENT_TIME;       -- TIMETZ type (with timezone)
SELECT LOCALTIME;          -- TIME type (without timezone)
SELECT LOCALTIMESTAMP;     -- TIMESTAMP without timezone

-- Practical implications
BEGIN;
-- NOW() captured at transaction start
INSERT INTO audit_log (action, tx_time) VALUES ('START', NOW());
-- 5 minutes later in same transaction
INSERT INTO audit_log (action, tx_time) VALUES ('LATER', NOW());
-- Both have SAME timestamp (transaction consistency)
COMMIT;

-- For actual wall-clock time during long operations
UPDATE big_table 
SET processed_at = CLOCK_TIMESTAMP()  -- Each row gets different time
WHERE ...;

-- Best practice: Use NOW() for business logic (consistent snapshots)
-- Use CLOCK_TIMESTAMP() for timing/debugging operations
```

### 9.3.2 Date/Time Arithmetic

```sql
-- Interval arithmetic
SELECT 
    NOW() + INTERVAL '1 day' as tomorrow,
    NOW() - INTERVAL '2 hours' as two_hours_ago,
    NOW() + INTERVAL '1 week 3 days' as future_date,
    DATE '2024-12-25' - DATE '2024-01-01' as days_between;  -- INTEGER result

-- Age calculation (returns INTERVAL)
SELECT 
    AGE(NOW(), birth_date) as age_interval,  -- Returns years, months, days
    EXTRACT(YEAR FROM AGE(NOW(), birth_date)) as age_years
FROM users;

-- Date truncation (grouping by periods)
SELECT 
    DATE_TRUNC('month', created_at) as month,
    COUNT(*) as orders
FROM orders
GROUP BY 1
ORDER BY 1;

-- Truncation levels: microsecond, millisecond, second, minute, hour, day, week, month, quarter, year, decade, century, millennium

-- Date part extraction
SELECT 
    created_at,
    EXTRACT(YEAR FROM created_at) as year,
    EXTRACT(MONTH FROM created_at) as month,
    EXTRACT(DAY FROM created_at) as day,
    EXTRACT(DOW FROM created_at) as day_of_week,  -- 0=Sunday, 6=Saturday
    EXTRACT(ISODOW FROM created_at) as iso_day,   -- 1=Monday, 7=Sunday
    EXTRACT(WEEK FROM created_at) as week_number, -- ISO week
    EXTRACT(EPOCH FROM created_at) as unix_timestamp  -- Seconds since 1970-01-01
FROM orders;

-- Generating series (time sequences)
SELECT generate_series(
    DATE '2024-01-01',
    DATE '2024-12-31',
    INTERVAL '1 day'
) as date_series;

-- Generate hourly slots for last 24 hours
SELECT generate_series(
    DATE_TRUNC('hour', NOW() - INTERVAL '24 hours'),
    DATE_TRUNC('hour', NOW()),
    INTERVAL '1 hour'
) as hour_slot;

-- Overlapping periods
SELECT 
    (DATE '2024-01-01', DATE '2024-06-01') OVERLAPS 
    (DATE '2024-05-01', DATE '2024-12-01') as overlaps;  -- TRUE

-- Boundary functions
SELECT 
    DATE_BIN(INTERVAL '15 minutes', event_time, TIMESTAMP '2000-01-01') as time_slot
FROM events;
-- Buckets timestamp into 15-minute intervals (PostgreSQL 14+)
```

### 9.3.3 Time Zone Handling

Time zone handling is a common source of bugs; storing everything in UTC is the industry standard.

```sql
-- Current timezone settings
SHOW timezone;  -- Display current session timezone

-- AT TIME ZONE conversions
SELECT 
    -- TIMESTAMPTZ -> TIMESTAMP (strip timezone, convert to zone)
    NOW() AT TIME ZONE 'America/New_York' as ny_local_time,
    
    -- TIMESTAMP -> TIMESTAMPTZ (interpret as being in zone, convert to UTC)
    TIMESTAMP '2024-01-15 12:00:00' AT TIME ZONE 'America/New_York' as with_tz;

-- Storing user preferences and converting
WITH user_event AS (
    SELECT 
        event_at,  -- Stored as TIMESTAMPTZ (UTC internally)
        'America/Los_Angeles' as user_tz
    FROM events
)
SELECT 
    event_at,  -- Displayed in session timezone
    event_at AT TIME ZONE user_tz as user_local_time
FROM user_event;

-- Daylight saving time awareness
SELECT 
    '2024-07-15 12:00:00'::timestamp AT TIME ZONE 'America/New_York' as summer_time,
    '2024-12-15 12:00:00'::timestamp AT TIME ZONE 'America/New_York' as winter_time;
-- Automatically handles DST transitions

-- Best practice: Store TIMESTAMPTZ (UTC), display in application layer
-- If you must store local time, store the timezone name separately

-- Timezone conversion table
SELECT * FROM pg_timezone_names 
WHERE name LIKE 'America/%';

-- Checking for DST
SELECT 
    is_dst
FROM pg_timezone_names
WHERE name = 'America/New_York';
```

### 9.3.4 Date Formatting (TO_CHAR/TO_DATE)

```sql
-- Formatting dates for display (TO_CHAR)
SELECT 
    TO_CHAR(NOW(), 'YYYY-MM-DD HH24:MI:SS') as iso_format,
    TO_CHAR(NOW(), 'Month DD, YYYY') as long_format,
    TO_CHAR(NOW(), 'Mon DDth, YYYY') as abbreviated,
    TO_CHAR(NOW(), 'DD/MM/YYYY') as european,
    TO_CHAR(NOW(), 'MM/DD/YYYY') as us,
    TO_CHAR(NOW(), 'YYYY-MM-DD"T"HH24:MI:SS"Z"') as iso8601_utc,
    TO_CHAR(amount, 'FM$999,999,999.00') as currency  -- Number formatting too
FROM data;

-- Parsing dates (TO_DATE/TO_TIMESTAMP)
SELECT 
    TO_DATE('2024-01-15', 'YYYY-MM-DD') as parsed_date,
    TO_TIMESTAMP('15-01-2024 14:30:00', 'DD-MM-YYYY HH24:MI:SS') as parsed_timestamp;

-- Format patterns reference:
-- YYYY: 4-digit year
-- MM: Month number (01-12)
-- DD: Day of month (01-31)
-- HH24: Hour (00-23)
-- MI: Minute (00-59)
-- SS: Second (00-59)
-- MS: Millisecond
-- TZ: Timezone abbreviation
-- FM: Fill mode (removes leading zeros)
```

## 9.4 JSON and JSONB Functions and Operators

### 9.4.1 JSON vs JSONB (When to Use Which)

```sql
-- JSON: Stored as exact text (preserves whitespace, key order, duplicates)
-- JSONB: Stored in binary format (normalized, indexed, no duplicates)

-- Insertion
INSERT INTO events (data) VALUES 
    ('{"name": "click", "timestamp": 123456}'),  -- JSON literal
    ('{"name": "click", "timestamp": 123456}'::jsonb);  -- JSONB cast

-- JSONB operators
SELECT 
    data->>'name' as text_value,           -- Get text (JSON -> text)
    data->'metadata' as json_object,       -- Get JSON object
    data->>'timestamp' as timestamp_text,  -- Extract as text
    data#>'{user,email}' as nested_value,  -- Path navigation (JSON)
    data#>>'{user,email}' as nested_text   -- Path navigation (text)
FROM events;

-- Containment and existence (JSONB only, uses indexes)
SELECT * FROM events WHERE data @> '{"status": "active"}';  -- Contains object
SELECT * FROM events WHERE data ? 'deleted_at';  -- Has key
SELECT * FROM events WHERE data ?| array['a', 'b'];  -- Has any of these keys
SELECT * FROM events WHERE data ?& array['a', 'b'];  -- Has all of these keys

-- Update JSONB
UPDATE events 
SET data = data || '{"status": "processed"}'::jsonb  -- Merge objects
WHERE id = 1;

-- Remove key
UPDATE events 
SET data = data - 'temporary_field'  -- Remove key
WHERE id = 1;

-- Remove multiple keys
UPDATE events 
SET data = data #- '{user,temp_data}'  -- Remove nested path
WHERE id = 1;
```

### 9.4.2 JSONB Aggregation and Construction

```sql
-- Build JSON objects from rows
SELECT 
    user_id,
    JSON_BUILD_OBJECT(
        'name', full_name,
        'email', email,
        'active', is_active
    ) as user_json
FROM users;

-- Build JSON arrays
SELECT 
    department_id,
    JSON_AGG(
        JSON_BUILD_OBJECT('id', user_id, 'name', full_name)
        ORDER BY full_name
    ) as team_members
FROM employees
GROUP BY department_id;

-- Alternative: JSONB_BUILD_OBJECT (returns JSONB instead of JSON)
SELECT JSONB_BUILD_OBJECT('key', 'value');

-- Row to JSON (entire row as JSON)
SELECT TO_JSONB(users) FROM users WHERE user_id = 1;

-- Array aggregation
SELECT 
    order_id,
    JSONB_AGG(
        JSONB_BUILD_OBJECT(
            'product', product_name,
            'qty', quantity,
            'price', unit_price
        ) ORDER BY line_item_id
    ) as items
FROM order_items
GROUP BY order_id;

-- Strip nulls (PostgreSQL 14+)
SELECT JSONB_STRIP_NULLS(data) FROM events;
-- Removes keys with null values

-- Pretty printing
SELECT JSONB_PRETTY(data) FROM events;
```

### 9.4.3 JSONB Path Queries (PostgreSQL 12+)

SQL/JSON path expressions provide standardized JSON querying.

```sql
-- Path query
SELECT 
    JSONB_PATH_QUERY(data, '$.user.email') as email
FROM events
WHERE JSONB_PATH_EXISTS(data, '$.user.premium');

-- Path match with vars
SELECT JSONB_PATH_QUERY(
    data, 
    '$.items[*] ? (@.price > $min)', 
    '{"min": 100}'
) as expensive_items
FROM orders;

-- Array length check
SELECT * FROM events 
WHERE JSONB_ARRAY_LENGTH(data->'tags') > 0;
```

## 9.5 Common Table Expressions (CTEs)

### 9.5.1 Non-Recursive CTEs (WITH clauses)

CTEs improve readability by breaking complex queries into named subqueries. They also serve as optimization fences (PostgreSQL executes them separately).

```sql
-- Basic CTE (readability)
WITH active_users AS (
    SELECT user_id, email, created_at
    FROM users
    WHERE status = 'active'
      AND deleted_at IS NULL
),
recent_orders AS (
    SELECT order_id, user_id, total, created_at
    FROM orders
    WHERE created_at > NOW() - INTERVAL '30 days'
)
SELECT 
    au.email,
    COUNT(ro.order_id) as order_count,
    COALESCE(SUM(ro.total), 0) as total_spent
FROM active_users au
LEFT JOIN recent_orders ro ON au.user_id = ro.user_id
GROUP BY au.user_id, au.email;

-- Multiple CTEs (data transformation pipeline)
WITH 
-- Step 1: Normalize raw data
cleaned_data AS (
    SELECT 
        TRIM(LOWER(email)) as normalized_email,
        NULLIF(phone, '') as clean_phone,
        created_at
    FROM raw_signups
    WHERE email IS NOT NULL
),
-- Step 2: Deduplicate
ranked AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY normalized_email ORDER BY created_at) as rn
    FROM cleaned_data
),
-- Step 3: Filter to first occurrence
deduped AS (
    SELECT * FROM ranked WHERE rn = 1
)
-- Final select
SELECT * FROM deduped;

-- CTEs with data modification (RETURNING)
WITH new_user AS (
    INSERT INTO users (email, full_name)
    VALUES ('new@example.com', 'New User')
    RETURNING user_id
),
audit_entry AS (
    INSERT INTO audit_log (user_id, action)
    SELECT user_id, 'USER_CREATED' FROM new_user
)
SELECT user_id FROM new_user;

-- Materialized CTEs (PostgreSQL 12+)
WITH 
    heavy_calculation AS MATERIALIZED (
        SELECT expensive_function(id), id FROM large_table
    )
SELECT * FROM heavy_calculation h1
JOIN heavy_calculation h2 ON h1.id = h2.id;
-- Forces materialization to avoid double execution
-- Without MATERIALIZED, Postgres might inline it twice
```

### 9.5.2 Recursive CTEs (Hierarchical Data)

Recursive CTEs traverse tree structures (org charts, bill of materials, threaded comments).

```sql
-- Organizational hierarchy (find all reports under manager)
WITH RECURSIVE org_tree AS (
    -- Anchor: Start with the VP
    SELECT 
        employee_id, 
        name, 
        manager_id, 
        0 as level,
        name::TEXT as path
    FROM employees
    WHERE employee_id = 100  -- VP ID
    
    UNION ALL
    
    -- Recursive: Find direct reports
    SELECT 
        e.employee_id,
        e.name,
        e.manager_id,
        ot.level + 1,
        ot.path || ' > ' || e.name
    FROM employees e
    INNER JOIN org_tree ot ON e.manager_id = ot.employee_id
    -- Must reference CTE itself (org_tree)
)
SELECT 
    REPEAT('  ', level) || name as indented_name,
    level,
    path
FROM org_tree
ORDER BY path;

-- Prevent infinite loops (cycles)
WITH RECURSIVE category_path AS (
    SELECT 
        id, 
        name, 
        parent_id,
        ARRAY[id] as path,  -- Track visited nodes
        0 as depth
    FROM categories
    WHERE parent_id IS NULL  -- Start with roots
    
    UNION ALL
    
    SELECT 
        c.id,
        c.name,
        c.parent_id,
        cp.path || c.id,
        cp.depth + 1
    FROM categories c
    JOIN category_path cp ON c.parent_id = cp.id
    WHERE NOT c.id = ANY(cp.path)  -- Prevent cycles (stop if already visited)
)
SELECT * FROM category_path;

-- Bill of materials (explode product components)
WITH RECURSIVE bom_explosion AS (
    -- Anchor: Final product
    SELECT 
        component_id,
        quantity as total_quantity,
        1 as level
    FROM bill_of_materials
    WHERE product_id = 'BIKE-001'
    
    UNION ALL
    
    -- Recursive: Get sub-components
    SELECT 
        bom.component_id,
        be.total_quantity * bom.quantity,
        be.level + 1
    FROM bill_of_materials bom
    JOIN bom_explosion be ON bom.product_id = be.component_id
)
SELECT 
    c.component_name,
    SUM(be.total_quantity) as total_required
FROM bom_explosion be
JOIN components c ON be.component_id = c.id
GROUP BY c.component_name;

-- Max recursion depth (safety)
WITH RECURSIVE deep_cte AS (
    SELECT 1 as n
    UNION ALL
    SELECT n + 1 FROM deep_cte WHERE n < 1000
)
SELECT * FROM deep_cte;
-- Default max_recursive_iterations = 1000
-- SET max_recursive_iterations = 100;  -- Limit for safety
```

### 9.5.3 CTE Performance Considerations

```sql
-- CTEs are optimization fences (usually)
-- This might be slower than subquery due to materialization
WITH cte_data AS (
    SELECT * FROM huge_table WHERE expensive_condition
)
SELECT * FROM cte_data a
JOIN cte_data b ON a.id = b.related_id;
-- CTE executes once, result materialized, then joined to itself

-- Alternative: Subquery (might be optimized better)
SELECT * FROM (
    SELECT * FROM huge_table WHERE expensive_condition
) a
JOIN (
    SELECT * FROM huge_table WHERE expensive_condition
) b ON a.id = b.related_id;
-- Planner might optimize this differently (predicate pushdown)

-- Use CTEs for:
-- 1. Readability in complex queries
-- 2. Recursive queries (only way)
-- 3. Data modification with RETURNING (chaining)
-- 4. When you need optimization fence (force specific plan)

-- Avoid CTEs for:
-- 1. Simple one-off subqueries (use subquery or JOIN)
-- 2. When you need predicate pushdown (unless MATERIALIZED)
```

## 9.6 LATERAL Joins

### 9.6.1 Understanding LATERAL

`LATERAL` allows subqueries in the `FROM` clause to reference columns from preceding tables—effectively a correlated subquery that can return multiple rows.

```sql
-- Top N per group (classic use case)
SELECT 
    u.user_id,
    u.email,
    recent_orders.*
FROM users u
LEFT JOIN LATERAL (
    SELECT order_id, total, created_at
    FROM orders o
    WHERE o.user_id = u.user_id  -- References outer table (u)
    ORDER BY created_at DESC
    LIMIT 3  -- Top 3 orders per user
) recent_orders ON true;

-- Without LATERAL (old way, inefficient)
SELECT u.*, o.*
FROM users u
LEFT JOIN orders o ON o.user_id = u.user_id
WHERE o.created_at = (
    SELECT MAX(created_at) 
    FROM orders o2 
    WHERE o2.user_id = u.user_id
);
-- Complex and doesn't handle ties well

-- LATERAL with functions (set-returning functions)
SELECT 
    u.user_id,
    recommendation.*
FROM users u,
LATERAL get_product_recommendations(u.user_id, 5) as recommendation;
-- Calls function for each user, returns 5 recommendations each

-- Multiple LATERAL joins
SELECT 
    c.category_id,
    c.name,
    top_product.product_name,
    stats.avg_price
FROM categories c
LEFT JOIN LATERAL (
    SELECT product_name, price
    FROM products p
    WHERE p.category_id = c.category_id
    ORDER BY sales_count DESC
    LIMIT 1
) top_product ON true
LEFT JOIN LATERAL (
    SELECT AVG(price) as avg_price
    FROM products p2
    WHERE p2.category_id = c.category_id
) stats ON true;

-- LATERAL VALUES (unnormalize arrays)
SELECT 
    u.user_id,
    u.email,
    permission
FROM users u,
LATERAL UNNEST(u.permissions) as t(permission);
-- Expands array column into rows
-- user_id=1, permissions=['read','write'] 
-- becomes two rows: (1, 'read'), (1, 'write')
```

### 9.6.2 LATERAL Performance and Optimization

```sql
-- Ensure lateral subqueries use indexes
CREATE INDEX idx_orders_user_created ON orders(user_id, created_at DESC);
-- Essential for "Top N per group" pattern to be fast

-- Alternative to LATERAL: Window functions (sometimes faster)
WITH ranked AS (
    SELECT 
        user_id,
        order_id,
        total,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC) as rn
    FROM orders
)
SELECT * FROM ranked WHERE rn <= 3;
-- Often faster than LATERAL for large datasets
-- LATERAL is better when outer table is small, inner table is large and indexed

-- When LATERAL is the only solution (complex logic)
SELECT 
    generate_series(start_date, end_date, '1 day'::interval) as date
FROM date_ranges;
-- Set-returning functions require LATERAL implicitly in FROM clause
```

---

## Chapter Summary

In this chapter, you learned:

1. **Conditional Logic**: Use `CASE` for conditional expressions (searched CASE for complex logic, simple CASE for equality); prefer `FILTER` clause over `CASE` in aggregates for readability; `COALESCE` for first non-NULL value (left-to-right evaluation); `NULLIF` to convert specific values to NULL (essential for safe division and data cleaning).

2. **String Manipulation**: Concatenate with `||` (ANSI standard) or `CONCAT` (NULL-safe); use `FORMAT` for type-safe templating; leverage `REGEXP_REPLACE` and `REGEXP_MATCHES` for pattern extraction; `STRING_AGG` for row-to-string aggregation with ordering; understand `OCTET_LENGTH` vs `CHAR_LENGTH` for UTF-8 data.

3. **Date/Time Operations**: Store all timestamps as `TIMESTAMPTZ` (UTC); use `NOW()` (transaction time) for consistency, `CLOCK_TIMESTAMP()` for wall-clock timing; `DATE_TRUNC` for period grouping; `AGE` for human-readable intervals; `GENERATE_SERIES` for time sequences; handle timezones with `AT TIME ZONE` conversions in application layer.

4. **JSON/JSONB**: Prefer `JSONB` for storage (binary, indexed, normalized); use `->>` for text extraction, `->` for JSON extraction; `@>` for containment queries (indexable); `||` for merging objects; `JSONB_BUILD_OBJECT` for construction; `JSONB_AGG` for aggregation; use SQL/JSON path (`JSONB_PATH_QUERY`) for complex navigation (PostgreSQL 12+).

5. **Common Table Expressions**: Use non-recursive CTEs for query organization and optimization fencing (MATERIALIZED prevents inlining); use recursive CTEs for hierarchical data with cycle detection (`ARRAY` tracking); remember CTEs act as optimization fences (execute once, materialize result).

6. **LATERAL Joins**: Enable correlated subqueries in FROM clause; essential for "Top N per group" patterns when combined with `LIMIT`; allows set-returning functions to reference outer tables; ensure proper indexes exist on lateral subquery join columns for performance; consider window functions as alternatives for large datasets.

---

**Next:** In Chapter 10, we will explore schema design fundamentals for OLTP systems—covering normalization strategies, key selection, handling many-to-many relationships, and implementing soft deletes and audit patterns that scale.